In [1]:
import numpy as np
import torch
from torch import nn
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import glob

### Constants

In [2]:
num_channels = 3
img_dimensions = (416, 416)

### Reading and Reshaping Images for Training

In [3]:
immature_img_paths = glob.glob('./train/immature/*')
mature_img_paths = glob.glob('./train/mature/*')

In [4]:
# immature_imgs_np = []
# mature_imgs_np = []
imgs_np = []
for imm_img_path in immature_img_paths:
    imgs_np.append(cv2.imread(imm_img_path))

for m_img_path in mature_img_paths:
    imgs_np.append(cv2.imread(m_img_path))

imgs_np = np.array(imgs_np)

In [5]:
# immature_imgs_np = immature_imgs_np.reshape(-1, num_channels, img_dimensions[0], img_dimensions[1])
# mature_imgs_np = mature_imgs_np.reshape(-1, num_channels, img_dimensions[0], img_dimensions[1])
imgs_np = imgs_np.reshape(-1, num_channels, img_dimensions[0], img_dimensions[1])

### Classifier Network Defn

In [6]:
class CataractClassifierNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv_net = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(),
        )

        self.linear_net = nn.Sequential(
                nn.Linear(in_features=5408, out_features=1024),
                nn.LeakyReLU(),
                nn.Linear(in_features=1024, out_features=512),
                nn.LeakyReLU(),
                nn.Linear(in_features=512, out_features=256),
                nn.LeakyReLU(),
                nn.Linear(in_features=256, out_features=128),
                nn.LeakyReLU(),
                nn.Linear(in_features=128, out_features=64),
                nn.LeakyReLU(),
                nn.Linear(in_features=64, out_features=32),
                nn.LeakyReLU(),
                nn.Linear(in_features=32, out_features=16),
                nn.LeakyReLU(),
                nn.Linear(in_features=16, out_features=8),
                nn.LeakyReLU(),
                nn.Linear(in_features=8, out_features=2),
                nn.Softmax(dim=-1)
            )
        

    def forward(self, x):
        batch_size = x.shape[0]
        x = self.conv_net(x)
        x = self.linear_net(x.reshape(batch_size, -1))
        return x

In [7]:
ccn = CataractClassifierNetwork()

In [22]:
# converting image array to image tensor 
imgs_tensor = torch.from_numpy(imgs_np)

### Defining Loss function and Optimizer

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ccn.parameters(), lr=3e-4)

In [10]:
len(immature_img_paths)

214

In [11]:
len(mature_img_paths)

196

In [12]:
Y_tensor = torch.zeros(len(immature_img_paths) + len(mature_img_paths))

In [13]:
Y_tensor[:len(immature_img_paths)] = 0
Y_tensor[len(immature_img_paths):] = 1

### Normalizing our Images

In [15]:
imgs_tensor = imgs_tensor / torch.max(imgs_tensor)

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(imgs_tensor, Y_tensor, test_size=0.1, train_size=0.9)

### Training

In [18]:
epochs = 50
for _ in range(epochs):
    loss = loss_fn(ccn(X_train.float()), Y_train.long())
    print(f"EPOCH {_ + 1}", loss)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

EPOCH 1 tensor(0.6928, grad_fn=<NllLossBackward0>)
EPOCH 2 tensor(0.6927, grad_fn=<NllLossBackward0>)
EPOCH 3 tensor(0.6927, grad_fn=<NllLossBackward0>)
EPOCH 4 tensor(0.6926, grad_fn=<NllLossBackward0>)
EPOCH 5 tensor(0.6926, grad_fn=<NllLossBackward0>)
EPOCH 6 tensor(0.6925, grad_fn=<NllLossBackward0>)
EPOCH 7 tensor(0.6925, grad_fn=<NllLossBackward0>)
EPOCH 8 tensor(0.6924, grad_fn=<NllLossBackward0>)
EPOCH 9 tensor(0.6923, grad_fn=<NllLossBackward0>)
EPOCH 10 tensor(0.6922, grad_fn=<NllLossBackward0>)
EPOCH 11 tensor(0.6920, grad_fn=<NllLossBackward0>)
EPOCH 12 tensor(0.6918, grad_fn=<NllLossBackward0>)
EPOCH 13 tensor(0.6916, grad_fn=<NllLossBackward0>)
EPOCH 14 tensor(0.6912, grad_fn=<NllLossBackward0>)
EPOCH 15 tensor(0.6906, grad_fn=<NllLossBackward0>)
EPOCH 16 tensor(0.6898, grad_fn=<NllLossBackward0>)
EPOCH 17 tensor(0.6889, grad_fn=<NllLossBackward0>)
EPOCH 18 tensor(0.6877, grad_fn=<NllLossBackward0>)
EPOCH 19 tensor(0.6859, grad_fn=<NllLossBackward0>)
EPOCH 20 tensor(0.683

In [19]:
def accuracy_score_softmax(y_true, y_pred_proba):
    """
    Calculate the accuracy of the classification model with softmax outputs.

    Parameters:
    y_true (array-like): True labels.
    y_pred_proba (array-like): Predicted probabilities from the softmax function.

    Returns:
    float: Accuracy score as a float between 0 and 1.
    """
    # Convert probabilities to predicted class labels
    y_pred = np.argmax(y_pred_proba, axis=1)
    # Calculate accuracy
    correct_predictions = np.sum(y_true == y_pred)
    total_predictions = len(y_true)
    accuracy = correct_predictions / total_predictions
    return accuracy

### Test Accuracy

In [21]:
accuracy_score_softmax(Y_test.detach().numpy(), ccn(X_test.float()).detach().numpy())

0.7804878048780488

### Train Accuracy

In [23]:
accuracy_score_softmax(Y_train.detach().numpy(), ccn(X_train.float()).detach().numpy())

0.8292682926829268